<!-- ---
reviewed_on: "2024-10-05"
--- -->

# Introduction to deep learning with PyTorch

[PyTorch](http://pytorch.org/) is a framework for building and training neural networks. PyTorch in a lot of ways behaves like the arrays you love from NumPy. These NumPy arrays, after all, are just tensors. PyTorch takes these tensors and makes it simple to move them to GPUs for the faster processing needed when training neural networks. It also provides a module that automatically calculates gradients (for backpropagation) and another module specifically for building neural networks. All together, PyTorch ends up being more coherent with Python and the NumPy/SciPy stack compared to TensorFlow and other frameworks.

## Neural networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

![neuronal network](./assets/01_01-neural_network.svg)

Mathematically this looks like

$$
\begin{align}
    y & = f(w_1 x_1 + w_2 x_2  + \dots + b) \\
    y & = f \left (\sum_{ i = 1 }^n { w_i x_i } + b \right )
\end{align}
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
    x_1,x_2 \cdots x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
            w_1 \\
            w_2 \\
            \vdots \\
            w_n
\end{bmatrix}
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

With the basics covered, it is time to explore how we can use PyTorch to build a simple neural network.

In [1]:
import torch

In [2]:
def activation(x):
	""" Sigmoid activation function

		Arguments
		---------
		x: torch.Tensor
	"""
	return 1 / (1 + torch.exp(-x))

In [ ]:
torch.manual_seed(7) # Set the random seed so things are predictable

features = torch.randn((1, 5))
weights = torch.randn_like(features)
bias = torch.randn((1, 1))

print(features, weights, bias)

- `features = torch.randn((1, 5))`: creates a tensor with shape `(1, 5)`, one row and five columns, that contains values randomly distributed according to the normal distribution with a mean of zero and standard deviation of one. 

- `weights = torch.randn_like(features)`: creates another tensor with the same shape as `features`, again containing values from a normal distribution.

- `bias = torch.randn((1, 1))`: creates a single value from a normal distribution.

### Without matrix multiplication

In [ ]:
y = activation(torch.sum(features * weights) + bias)
print(y)

### With matrix multiplication

- `weights.reshape(a, b)`: will return a new tensor with the same data as `weights` with size `(a, b)` sometimes, and sometimes a clone, as in it copies the data to another part of memory.

- `weights.resize_(a, b)`: returns the same tensor with a different shape. However, if the new shape results in fewer elements than the original tensor, some elements will be removed from the tensor (but not from memory). If the new shape results in more elements than the original tensor, new elements will be uninitialized in memory.

    > It is performed **in-place**.

- `weights.view(a, b)`: will return a new tensor with the same data as `weights` with size `(a, b)`.

In [ ]:
weights.shape
weights = weights.reshape(5, 1) # weights.view(5,1)

y = activation(torch.mm(features, weights) + bias)
print(y)

### Stack them up!

That is how you can calculate the output for a single neuron. The real power of this algorithm happens when you start stacking these individual units into layers and stacks of layers, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

![neuronal network](./assets/01_02-neural_network.svg)

The first layer shown on the bottom here are the inputs, understandably called the **input layer**. The middle layer is called the **hidden layer**, and the final layer (on the right) is the **output layer**. We can express this network mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \left (f_1 \left (\vec{x} W_1 \right ) W_2 \right )
$$

In [ ]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W_1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W_2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B_1 = torch.randn((1, n_hidden))
B_2 = torch.randn((1, n_output))

In [ ]:
h = activation(torch.mm(features, W_1) + B_1)
output = activation(torch.mm(h, W_2) + B_2)

The number of hidden units is a parameter of the network, often called a **hyperparameter** to differentiate it from the weights and biases parameters. As you will see later when we discuss training a neural network, the more hidden units a network has, and the more layers, the better able it is to learn from data and make accurate predictions.

## NumPy to Torch and back

Special bonus section! PyTorch has a great feature for converting between NumPy arrays and Torch tensors. To create a tensor from a NumPy array, use `torch.from_numpy()`. To convert a tensor to a NumPy array, use the `.numpy()` method.

In [ ]:
import numpy as np


np.set_printoptions(precision=8)
a = np.random.rand(4,3)
a

In [ ]:
torch.set_printoptions(precision=8)
b = torch.from_numpy(a)
b

In [ ]:
b.numpy()

The memory is shared between the NumPy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [ ]:
b.mul_(2)
a